In [ ]:
# 若是在 Colab，第一次先安裝 unzip（大多數映像已內建）
!apt-get -q install -y unzip



ZIP_PATH = "/content/drive/MyDrive/datasetWith6000FLUX1Dev_v10.zip"    # 你的 zip 檔
DEST     = "/content/drive/MyDrive/data/raw/fake"        # 解壓目標

!mkdir -p "$DEST"
!unzip -q "$ZIP_PATH" -d "$DEST"     # -q 安靜模式

# （可選）解壓後刪掉 zip
# !rm "$ZIP_PATH"

print("✅ 已解壓完成 →", DEST)


Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive
replace /content/drive/MyDrive/data/raw/fake/6k-flux-images/ComfyUI_07476_.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ✅ 已解壓完成 → /content/drive/MyDrive/data/raw/fake


In [ ]:
# ⚠️ 在新 cell 執行即可
!apt-get -q install -y p7zip-full  # 安裝 7-Zip CLI（Colab 尚未安裝時）


# 2. 路徑設定 ── 修改成你的檔案與目標資料夾
ARCHIVE = "/content/tmp/fake.7z"        # .7z 檔
DEST    = "/content/tmp"           # 圖片要放的位置

# 3. 確定目的資料夾存在
import os, pathlib
pathlib.Path(DEST).mkdir(parents=True, exist_ok=True)

# 4. 解壓縮到目標資料夾
!7z x -y -o"$DEST" "$ARCHIVE"

# 5. （可選）解壓完成後刪除壓縮檔，釋放 Drive 空間
#    如果想保留 .7z，註解掉下面這一行即可
!rm "$ARCHIVE"

print("✅ 解壓完畢！圖片都在", DEST)


Reading package lists...
Building dependency tree...
Reading state information...
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/tmp/                       1 file, 3935202486 bytes (3753 MiB)

Extracting archive: /content/tmp/fake.7z
--
Path = /content/tmp/fake.7z
Type = 7z
Physical Size = 3935202486
Headers Size = 619203
Method = LZMA:23
Solid = +
Blocks = 1

  0%      0% - fake/000000.jpg                        0% 51 - fake/000051.jpg

In [3]:
# ╔═╗  batch unzip Drive → Colab tmp → Drive  ════════════════════════════╗
import pathlib, shutil, zipfile, tqdm, os

# === 請依需求修改這三行 ===
ZIP_DIR     = "/home/yaya/aigen/data/zips/flux"      # 存放多個 .zip 的資料夾
EXTRACT_DIR = "/home/yaya/aigen/data/raw/fake/flux"  # 解壓後要放哪
KEEP_ZIP    = True      # True ➜ 解壓完保留 zip；False ➜ 刪除 zip

# ───────────────────────────────────────────────────────────────────────

zip_dir     = pathlib.Path(ZIP_DIR)
extract_dir = pathlib.Path(EXTRACT_DIR); extract_dir.mkdir(parents=True, exist_ok=True)

zips = sorted(zip_dir.glob("*.zip"))
print(f"共偵測到 {len(zips)} 個 zip 檔，需要解壓…")

for z in tqdm.tqdm(zips, unit="zip"):
    marker = extract_dir / f".done_{z.stem}"    # 標記檔：若存在代表該 zip 已解壓
    if marker.exists():
        continue

    # 1️⃣ 複製 zip 到 /tmp，減少 Drive ↔ Colab 的重複 I/O
    tmp_zip = pathlib.Path("/tmp") / z.name
    shutil.copy2(z, tmp_zip)

    # 2️⃣ 在 /tmp 解壓
    with zipfile.ZipFile(tmp_zip) as zf:
        zf.extractall(path=extract_dir)

    # 3️⃣ 標記已完成
    marker.touch()

    # 4️⃣ 清理
    tmp_zip.unlink()
    if not KEEP_ZIP:
        z.unlink()          # 刪掉原本 Drive 上的 zip

print("✅ 所有 zip 已解壓完成！")
# ╚══════════════════════════════════════════════════════════════════════╝


共偵測到 5 個 zip 檔，需要解壓…


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:11<00:00, 14.33s/zip]

✅ 所有 zip 已解壓完成！
